In [1]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer

In [3]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = Path("datasets/housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"    
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not housing_path.exists():
        housing_path.mkdir(parents=True)
    tgz_path = housing_path / "housing.tgz"
    urllib.request.urlretrieve(housing_url, tgz_path)
    with tarfile.open(tgz_path) as housing_tgz:
        housing_tgz.extractall(path=housing_path)   
        
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = housing_path / "housing.csv"
    return pd.read_csv(csv_path)    

In [4]:
fetch_housing_data()
housing = load_housing_data()

C:\Users\rjraj\AppData\Local\Temp\ipykernel_8344\52364456.py:10: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  housing_tgz.extractall(path=housing_path)
